In [1]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
from evaluate import load
import pandas as pd
import numpy as np
from datasets import load_dataset
import torch
import json

c:\Users\carlos\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "deepset/deberta-v3-large-squad2"

In [3]:
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

In [4]:
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
model.eval()    

DebertaV2ForQuestionAnswering(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 1024, padding_idx=0)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0): DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=T

In [6]:
raw_datasets = load_dataset("squad_v2")

print("Context: ", raw_datasets["validation"][0]["context"])
print("Question: ", raw_datasets["validation"][0]["question"])
print("Answer: ", raw_datasets["validation"][0]["answers"])

Found cached dataset squad_v2 (C:/Users/carlos/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)
100%|██████████| 2/2 [00:00<00:00, 141.80it/s]

Context:  The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.
Question:  In what country is Normandy located?
Answer:  {'text': ['France', 'France', 'France', 'France'], 'answer_start': [159, 159, 159, 159]}


In [7]:
raw_datasets["validation"].filter(lambda x: len(x["answers"]["text"]) != 1)
print(raw_datasets["validation"][0]["answers"])
print(raw_datasets["validation"][2]["answers"])

Loading cached processed dataset at C:\Users\carlos\.cache\huggingface\datasets\squad_v2\squad_v2\2.0.0\09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d\cache-a729e31eeecedd01.arrow


{'text': ['France', 'France', 'France', 'France'], 'answer_start': [159, 159, 159, 159]}
{'text': ['Denmark, Iceland and Norway', 'Denmark, Iceland and Norway', 'Denmark, Iceland and Norway', 'Denmark, Iceland and Norway'], 'answer_start': [256, 256, 256, 256]}


In [8]:
def get_predictions(question, context):
    QA_input = {
        'question': question,
        'context': context
    }
    return nlp(QA_input)

In [ ]:
validation_ds = raw_datasets["validation"]
predictions = {}
for i in range(10) :
    item = validation_ds[i]
    question = item["question"]
    context = item["context"]
    prediction = get_predictions(question, context)
    predictions[item["id"]] = prediction

with open("predictions.json", "w") as outfile:
    json.dump(predictions, outfile)